<a href="https://colab.research.google.com/github/nbPittigiani/Ev_Analytics/blob/main/Eletric_Vehicle_Analytics_Modelagem_Preditiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Introdução**

Este notebook documenta o processo completo de desenvolvimento de um modelo de machine learning para prever o valor de revenda de veículos elétricos, representado pela variável `Resale_Value_USD`. Utilizando um conjunto de dados que abrange especificações técnicas, dados de uso e custos, o projeto visa não apenas criar um modelo preditivo acurado, mas também identificar as características mais influentes na determinação do preço de revenda.

---

O estudo foi estruturado em quatro etapas principais:

1.  **Análise Exploratória de Dados (EDA):** Uma investigação aprofundada para entender a estrutura dos dados, a distribuição das variáveis e as relações preliminares entre elas (vide notebook - [EDA](https://github.com/nbPittigiani/Ev_Analytics/blob/main/Eletric_Vehicle_Analytics.ipynb)).
2.  **Pré-processamento e Engenharia de Features:** Preparação dos dados para a modelagem, incluindo o tratamento de variáveis, normalização e a criação de novas features com maior poder preditivo.
3.  **Modelagem Preditiva:** Treinamento e avaliação de múltiplos algoritmos de regressão, partindo de um modelo linear como baseline e avançando para modelos de ensemble mais complexos, como Random Forest e Gradient Boosting.
4.  **Análise e Interpretação:** Avaliação final da performance dos modelos e uma análise da importância das features para extrair insights sobre quais fatores mais impactam o valor de revenda de um veículo elétrico.

O objetivo final é fornecer um modelo funcional e uma análise clara dos principais impulsionadores de valor no mercado de veículos elétricos, com base nos dados disponíveis.

In [58]:
import pandas as pd
import numpy as np
from datetime import datetime
from typing import List

# --- Módulos do Scikit-learn para Pré-processamento ---
# Função para dividir o dataset em conjuntos de treino e teste.
from sklearn.model_selection import train_test_split
# Classes para padronizar features numéricas e codificar as categóricas.
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# Ferramenta para aplicar diferentes transformadores a diferentes colunas do dataset.
from sklearn.compose import ColumnTransformer
# Classe para criar um Pipeline.
from sklearn.pipeline import Pipeline

# --- Módulos do Scikit-learn para Modelagem e Avaliação ---
# Modelo de Regressão Linear.
from sklearn.linear_model import LinearRegression
# Importar o regressor Random Forest
from sklearn.ensemble import RandomForestRegressor
# Importar o regressor Gradient Boosting
from sklearn.ensemble import GradientBoostingRegressor
# Funções para calcular as métricas de erro e performance do modelo de regressão.
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [59]:
filename = 'electric_vehicle_analytics.csv'
ev_df = pd.read_csv(filename)
ev_df.head()

,Vehicle_ID,Make,Model,Year,Region,Vehicle_Type,Battery_Capacity_kWh,Battery_Health_%,Range_km,Charging_Power_kW,...,Max_Speed_kmh,Acceleration_0_100_kmh_sec,Temperature_C,Usage_Type,CO2_Saved_tons,Maintenance_Cost_USD,Insurance_Cost_USD,Electricity_Cost_USD_per_kWh,Monthly_Charging_Cost_USD,Resale_Value_USD
0,1,Nissan,Leaf,2021,Asia,SUV,101.7,75.5,565,153.6,...,233,8.10,-9.0,Personal,14.13,969,843,0.30,375.55,26483
1,2,Nissan,Leaf,2020,Australia,Sedan,30.1,99.8,157,157.2,...,221,9.83,1.6,Personal,19.41,1157,1186,0.25,532.02,11287
2,3,Hyundai,Kona Electric,2021,North America,SUV,118.5,84.0,677,173.6,...,138,3.60,1.5,Fleet,29.39,291,1890,0.26,1291.68,34023
3,4,Audi,Q4 e-tron,2022,Europe,Hatchback,33.1,97.3,149,169.3,...,192,8.97,12.5,Fleet,6.96,401,2481,0.33,234.44,14398
4,5,Tesla,Model 3,2022,Australia,Truck,81.3,85.6,481,212.8,...,189,7.03,-3.0,Commercial,2.06,214,2336,0.10,32.61,23033


## **1.1. Transformação Logarítmica**

Durante a análise exploratória, observou-se que variáveis como custos e tempos frequentemente apresentam uma **distribuição assimétrica à direita**.

Para mitigar o impacto dessa assimetria nos modelos de regressão, foi aplicada uma transformação logarítmica nas colunas `Monthly_Cost` e `Charging_Time`. Os benefícios e a justificativa detalhada para esta operação foram abordados na seção de Análise Exploratória de Dados [EDA](https://github.com/nbPittigiani/Ev_Analytics/blob/main/Eletric_Vehicle_Analytics.ipynb) deste projeto.

A utilização das versões logarítmicas dessas variáveis contribui para a construção de um modelo preditivo mais robusto e confiável.

In [60]:
log_df = ev_df.copy()
log_df['log_Charging_Time_hr'] = np.log1p(log_df['Charging_Time_hr'])
log_df['log_Monthly_Charging_Cost_USD'] = np.log1p(log_df['Monthly_Charging_Cost_USD'])
log_df = log_df.drop(columns=['Charging_Time_hr', 'Monthly_Charging_Cost_USD'])
log_df.head()

,Vehicle_ID,Make,Model,Year,Region,Vehicle_Type,Battery_Capacity_kWh,Battery_Health_%,Range_km,Charging_Power_kW,...,Acceleration_0_100_kmh_sec,Temperature_C,Usage_Type,CO2_Saved_tons,Maintenance_Cost_USD,Insurance_Cost_USD,Electricity_Cost_USD_per_kWh,Resale_Value_USD,log_Charging_Time_hr,log_Monthly_Charging_Cost_USD
0,1,Nissan,Leaf,2021,Asia,SUV,101.7,75.5,565,153.6,...,8.10,-9.0,Personal,14.13,969,843,0.30,26483,0.598837,5.931051
1,2,Nissan,Leaf,2020,Australia,Sedan,30.1,99.8,157,157.2,...,9.83,1.6,Personal,19.41,1157,1186,0.25,11287,0.239017,6.278559
2,3,Hyundai,Kona Electric,2021,North America,SUV,118.5,84.0,677,173.6,...,3.60,1.5,Fleet,29.39,291,1890,0.26,34023,0.609766,7.164473
3,4,Audi,Q4 e-tron,2022,Europe,Hatchback,33.1,97.3,149,169.3,...,8.97,12.5,Fleet,6.96,401,2481,0.33,14398,0.223144,5.461456
4,5,Tesla,Model 3,2022,Australia,Truck,81.3,85.6,481,212.8,...,7.03,-3.0,Commercial,2.06,214,2336,0.10,23033,0.357674,3.514824


---

# **2. Engenharia de Features**

Para aprimorar o poder preditivo do modelo, foram realizadas etapas de engenharia e transformação de features.

**Features**

Com o objetivo de extrair informações mais contextuais e representativas dos dados brutos, quatro novas features foram criadas:

1.  **`Vehicle_Age`**: Calculada a partir do ano de fabricação (`Year`), esta variável representa a depreciação de forma mais direta.
2.  **`Battery_Efficiency_km/kWh`**: Mede a autonomia do veículo por kWh, servindo como um indicador direto de eficiência energética.
3.  **`Battery_Degradation_%`**: Representa o percentual de "desgaste" da bateria (`100 - Battery_Health_%`), que pode ter uma correlação negativa mais clara com o valor de revenda.
4.  **`Avg_km_per_Year`**: Normaliza a quilometragem total pela idade do veículo, oferecendo uma medida da intensidade de uso anual.

In [61]:
feature_df = log_df.copy()
atual_year: int = datetime.now().year

In [62]:
# Criar 'Vehicle_Age'
feature_df['Vehicle_Age'] = atual_year - feature_df['Year']

# Criar 'Battery_Efficiency_km/kWh'
feature_df['Battery_Efficiency_km/kWh'] = feature_df['Range_km'] / feature_df['Battery_Capacity_kWh']

# Criar 'Battery_Degradation_%'
feature_df['Battery_Degradation_%'] = 100 - feature_df['Battery_Health_%']

# Criar 'Avg_km_per_Year'
# Soma 1 à idade do veículo para evitar divisões por zero
idade_calculo: pd.Series = feature_df['Vehicle_Age'] + 1
feature_df['Avg_km_per_Year'] = feature_df['Mileage_km'] / idade_calculo

feature_df.head()

,Vehicle_ID,Make,Model,Year,Region,Vehicle_Type,Battery_Capacity_kWh,Battery_Health_%,Range_km,Charging_Power_kW,...,Maintenance_Cost_USD,Insurance_Cost_USD,Electricity_Cost_USD_per_kWh,Resale_Value_USD,log_Charging_Time_hr,log_Monthly_Charging_Cost_USD,Vehicle_Age,Battery_Efficiency_km/kWh,Battery_Degradation_%,Avg_km_per_Year
0,1,Nissan,Leaf,2021,Asia,SUV,101.7,75.5,565,153.6,...,969,843,0.30,26483,0.598837,5.931051,4,5.555556,24.5,23545.40
1,2,Nissan,Leaf,2020,Australia,Sedan,30.1,99.8,157,157.2,...,1157,1186,0.25,11287,0.239017,6.278559,5,5.215947,0.2,26955.00
2,3,Hyundai,Kona Electric,2021,North America,SUV,118.5,84.0,677,173.6,...,291,1890,0.26,34023,0.609766,7.164473,4,5.713080,16.0,48986.20
3,4,Audi,Q4 e-tron,2022,Europe,Hatchback,33.1,97.3,149,169.3,...,401,2481,0.33,14398,0.223144,5.461456,3,4.501511,2.7,14498.75
4,5,Tesla,Model 3,2022,Australia,Truck,81.3,85.6,481,212.8,...,214,2336,0.10,23033,0.357674,3.514824,3,5.916359,14.4,4296.25


### **2.1. Seleção de Features**

Antes da etapa de modelagem, foi realizado um processo de seleção de features para otimizar o conjunto de dados. O objetivo foi remover variáveis não informativas, redundantes ou que pudessem adicionar complexidade desnecessária ao modelo.

As seguintes colunas foram removidas:

* **`Vehicle_ID`**: Por ser um identificador único, não possui valor preditivo para o modelo.
* **`Year`**: Removida para evitar redundância, uma vez que sua informação foi capturada de forma mais direta pela feature `Vehicle_Age`.
* **`Battery_Health_%`**: Descartada por ser redundante. A feature `Battery_Degradation_%` representa a mesma informação, focando no conceito de desgaste, que tem uma relação mais intuitiva com a depreciação.
* **`Mileage_km`**: Substituída pela `Avg_km_per_Year`. Esta nova métrica normaliza a quilometragem pela idade do veículo, fornecendo um indicador mais robusto sobre a intensidade de uso.
* **`Model`**: Removida preventivamente devido à sua **alta cardinalidade**. A sua inclusão via *One-Hot Encoding* aumentaria excessivamente a dimensionalidade do dataset, o que poderia prejudicar a performance e o tempo de treinamento do modelo.

Este processo de seleção resultou em um conjunto de dados mais enxuto e focado, contendo as features mais relevantes para a previsão do valor de revenda.

In [63]:
col_to_drop = ['Vehicle_ID', 'Year', 'Battery_Health_%', 'Mileage_km', 'Model']
feature_df = feature_df.drop(columns=col_to_drop)
feature_df.head()


,Make,Region,Vehicle_Type,Battery_Capacity_kWh,Range_km,Charging_Power_kW,Charge_Cycles,Energy_Consumption_kWh_per_100km,Avg_Speed_kmh,Max_Speed_kmh,...,Maintenance_Cost_USD,Insurance_Cost_USD,Electricity_Cost_USD_per_kWh,Resale_Value_USD,log_Charging_Time_hr,log_Monthly_Charging_Cost_USD,Vehicle_Age,Battery_Efficiency_km/kWh,Battery_Degradation_%,Avg_km_per_Year
0,Nissan,Asia,SUV,101.7,565,153.6,1438,12.76,53.4,233,...,969,843,0.30,26483,0.598837,5.931051,4,5.555556,24.5,23545.40
1,Nissan,Australia,Sedan,30.1,157,157.2,1056,15.79,58.0,221,...,1157,1186,0.25,11287,0.239017,6.278559,5,5.215947,0.2,26955.00
2,Hyundai,North America,SUV,118.5,677,173.6,1497,24.34,69.4,138,...,291,1890,0.26,34023,0.609766,7.164473,4,5.713080,16.0,48986.20
3,Audi,Europe,Hatchback,33.1,149,169.3,1613,14.70,42.9,192,...,401,2481,0.33,14398,0.223144,5.461456,3,4.501511,2.7,14498.75
4,Tesla,Australia,Truck,81.3,481,212.8,1078,22.77,97.6,189,...,214,2336,0.10,23033,0.357674,3.514824,3,5.916359,14.4,4296.25


---

# **3. Conj Treino e Teste**

Para a construção do modelo preditivo, o conjunto de dados foi dividido em duas partes: a matriz de features (variáveis independentes, `X`) e o vetor alvo (variável dependente, `y`). A variável `Resale_Value_USD` foi definida como o alvo da previsão.

Posteriormente, os dados foram segmentados em conjuntos de treino e teste, utilizando uma proporção de 80% para treino e 20% para teste. Esta divisão é fundamental para o processo de modelagem, pois permite treinar o modelo com a maioria dos dados e, em seguida, avaliar sua performance de generalização em um conjunto de dados "não vistos" (o conjunto de teste). Para garantir a reprodutibilidade dos resultados, um `random_state` foi fixado durante a divisão.


In [64]:
# Variável Alvo
TARGET_VARIABLE = 'Resale_Value_USD'

# Matriz de Features
X = feature_df.drop(columns=[TARGET_VARIABLE])
# Vetor Alvo
y = feature_df[TARGET_VARIABLE]

In [65]:
# Dividinado conjunto Treino/Teste
# 20% dos dados serao utilizados para teste
# random_state=42 garante reprodutibilidade do teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f'X_train: {X_train.shape}\nX_test: {X_test.shape}')

X_train: (2400, 23)
X_test: (600, 23)


## **3.1. Investigando a Multicolinearidade:**

A forma padrão de medir a multicolinearidade é calcular o **Fator de Inflação de Variância (VIF)** para cada feature. Uma regra geral é que um VIF acima de 5 ou 10 indica uma correlação problemática que precisa ser investigada.

Caso a multicolinearidade seja confirmada, podemos experimentar modelos que são naturalmente mais robustos a ela, como **Random Forest** ou **Gradient Boosting**, que são imunes a este problema.

In [66]:
# Importar a biblioteca necessária para testar multicolinearidade
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Selecionar apenas as features numéricas do conjunto de treino original
numeric_features_df = X_train.select_dtypes(include='number')

# Adicionar uma coluna de constante (intercepto), necessária para o cálculo do VIF
numeric_features_df['intercept'] = 1

# Calcular o VIF para cada feature numérica
# Criamos um DataFrame para armazenar os resultados
vif_data = pd.DataFrame()
vif_data["feature"] = [col for col in numeric_features_df.columns if col != 'intercept']
vif_data["VIF"] = [variance_inflation_factor(numeric_features_df.values, i)
                   for i in range(len(numeric_features_df.columns))
                   if numeric_features_df.columns[i] != 'intercept']

vif_data.sort_values('VIF', ascending=False).reset_index(drop=True)

,feature,VIF
0,Range_km,86.919269
1,Battery_Capacity_kWh,77.888024
2,Battery_Efficiency_km/kWh,9.302239
3,CO2_Saved_tons,8.223250
4,log_Monthly_Charging_Cost_USD,7.988227
5,Avg_km_per_Year,4.168960
6,log_Charging_Time_hr,4.062518
7,Charging_Power_kW,3.566208
8,Vehicle_Age,2.535480
9,Electricity_Cost_USD_per_kWh,2.331227


## **3.2. Análise VIF**

A análise de multicolinearidade com VIF revelou uma forte correlação entre as principais features do modelo, especialmente entre `Range_km` e `Battery_Capacity_kWh`. Diante desse diagnóstico, optou-se por explorar algoritmos mais robustos a este fenômeno.

> O modelo de Regressão Linear será mantido neste notebook para fins comparativos, servindo como um *baseline* de performance.

---

# **4. **Pipeline** de Pré-Processamento**

Para garantir que as transformações de dados sejam aplicadas de maneira consistente tanto no conjunto de treino quanto no de teste, foi construído um pipeline de pré-processamento utilizando o `ColumnTransformer` do Scikit-learn. Esta abordagem automatiza o tratamento necessário para cada tipo de variável.

O pipeline foi configurado para executar as seguintes operações:

1.  **Para as features numéricas**: Foi aplicado o `StandardScaler`, que padroniza os dados, removendo a média e escalonando para a variância unitária. Este passo é crucial para modelos sensíveis à escala das variáveis, como a Regressão Linear.
2.  **Para as features categóricas**: Utilizou-se o `OneHotEncoder`, que converte as variáveis nominais em um formato numérico binário (dummy variables). O parâmetro `handle_unknown='ignore'` foi definido para evitar erros caso o modelo encontre categorias no conjunto de teste que não estavam presentes no conjunto de treino.

Essa estrutura centraliza todo o pré-processamento em um único objeto, simplificando o fluxo de trabalho e prevenindo o vazamento de dados (data leakage).


In [67]:
def build_preprocessor(X: pd.DataFrame) -> ColumnTransformer:
    """
    Constrói o ColumnTransformer para o pré-processamento dos dados.

    A função identifica automaticamente as colunas numéricas e categóricas
    e cria um pipeline que aplica StandardScaler às numéricas e
    OneHotEncoder às categóricas.

    Parâmetros
    ----------
    X : pd.DataFrame
        O DataFrame de features.

    Retorno
    -------
    ColumnTransformer
        O objeto de pré-processamento configurado.
    """
    # Identificando features numéricas e categóricas
    numeric_features: List[str] = X.select_dtypes(include=np.number).columns.tolist()
    categorical_features: List[str] = X.select_dtypes(exclude=np.number).columns.tolist()

    print(f"Features Numéricas: {numeric_features}\nFeatures Categóricas: {categorical_features}")

    # Criando os pipelines para cada tipo de feature
    # Padroniza os dados (média 0, desvio padrão 1)
    numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

    # Converte categorias em colunas binárias (0s e 1s)
    # handle_unknown='ignore' evita erros se uma categoria nova aparecer nos dados de teste
    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

    # Combinando os pipelines com ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='passthrough'
    )

    return preprocessor

# Construindo o pré-processador
preprocessor = build_preprocessor(X_train)

Features Numéricas: ['Battery_Capacity_kWh', 'Range_km', 'Charging_Power_kW', 'Charge_Cycles', 'Energy_Consumption_kWh_per_100km', 'Avg_Speed_kmh', 'Max_Speed_kmh', 'Acceleration_0_100_kmh_sec', 'Temperature_C', 'CO2_Saved_tons', 'Maintenance_Cost_USD', 'Insurance_Cost_USD', 'Electricity_Cost_USD_per_kWh', 'log_Charging_Time_hr', 'log_Monthly_Charging_Cost_USD', 'Vehicle_Age', 'Battery_Efficiency_km/kWh', 'Battery_Degradation_%', 'Avg_km_per_Year']
Features Categóricas: ['Make', 'Region', 'Vehicle_Type', 'Usage_Type']


In [68]:
# Aplicando o pré-processador
# fit_transform() nos dados de treino para aprender as transformações e aplicá-las
X_train_processed = preprocessor.fit_transform(X_train)

# transform() nos dados de teste para aplicar as mesmas transformações aprendidas
X_test_processed = preprocessor.transform(X_test)

print(f"\nFormato dos dados de treino processados: {X_train_processed.shape}")
print(f"Formato dos dados de teste processados: {X_test_processed.shape}")
print("\nExemplo da primeira linha de dados de treino após o pré-processamento:")
print(X_train_processed[0])


Formato dos dados de treino processados: (2400, 40)
Formato dos dados de teste processados: (600, 40)

Exemplo da primeira linha de dados de treino após o pré-processamento:
[-1.04493928 -1.17786275  0.31377171 -1.28791307 -0.1033082  -1.04199079
 -1.28555045  1.59099839 -0.03901839 -0.40635279  1.12568     1.58823836
 -1.74843593 -0.77368732 -0.99572051  0.1770813  -0.98461965  0.30932134
 -0.49861839  0.          0.          0.          0.          0.
  0.          1.          0.          0.          0.          0.
  1.          0.          0.          0.          0.          1.
  0.          1.          0.          0.        ]


## **4.1. Resultado**

A aplicação do pipeline de pré-processamento transformou os conjuntos de dados de treino e teste, resultando em uma matriz final com 40 colunas para ambos. O aumento na dimensionalidade é um resultado direto da aplicação do `OneHotEncoder`, que expandiu as variáveis categóricas em múltiplas colunas binárias.

Com os dados agora em um formato numérico, homogêneo e devidamente escalonado, o conjunto está pronto para ser utilizado no treinamento do modelo.

---

# **5. Modelo de Regressão Linear**

A primeira etapa da modelagem preditiva consiste em estabelecer um *baseline* de performance. Para este fim, foi escolhido o modelo de **Regressão Linear**. Por sua simplicidade e interpretabilidade, ele é ideal para criar um ponto de partida claro, permitindo avaliar a eficácia de modelos mais complexos posteriormente.

Para garantir a correta aplicação do pré-processamento, o modelo foi integrado a um `Pipeline` que primeiro executa as transformações de dados (padronização e encoding) e, em seguida, treina o regressor. O objetivo é treinar o modelo com os dados de treino e avaliá-lo no conjunto de teste para obter as métricas iniciais de performance.


In [69]:
regression_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
    ])

In [70]:
# Treinar o pipeline completo com os dados de treino
regression_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Battery_Capacity_kWh',
                                                   'Range_km',
                                                   'Charging_Power_kW',
                                                   'Charge_Cycles',
                                                   'Energy_Consumption_kWh_per_100km',
                                                   'Avg_Speed_kmh',
                                                   'Max_Speed_kmh',
                                                   'Acceleration_0_100_kmh_sec',
                                                   'Temperature_C',
                                                   'CO2_Saved_tons',
                                                   'Maintena...
                                                   'Insurance_Cost_USD',
                                                   'Electricity_Cost_USD_per_kWh',
                                                   'log_Charging_Time_hr',
                                                   'log_Monthly_Charging_Cost_USD',
                                                   'Vehicle_Age',
                                                   'Battery_Efficiency_km/kWh',
                                                   'Battery_Degradation_%',
                                                   'Avg_km_per_Year']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Make', 'Region',
                                                   'Vehicle_Type',
                                                   'Usage_Type'])])),
                ('regressor', LinearRegression())])

## **5.1. Métricas de Avaliação do Modelo**

A performance foi avaliada utilizando três métricas para problemas de regressão:

* **Erro Absoluto Médio (MAE)**: Indica, em média, o quão distantes as previsões do modelo estão dos valores reais, na mesma unidade da variável alvo (USD).
* **Erro Quadrático Médio (MSE)**: Similar ao MAE, mas penaliza erros maiores de forma mais significativa por elevá-los ao quadrado.
* **Coeficiente de Determinação (R²)**: Mede a proporção da variância na variável alvo que é explicada pelas features do modelo. Um valor de 100% indicaria um ajuste perfeito.

Essas métricas fornecem uma avaliação quantitativa da capacidade do modelo de generalizar para novos dados e estabelecem a linha de base de performance que modelos mais complexos tentarão superar.

In [71]:
# Fazer a previsão nos dados de teste
y_pred = regression_pipeline.predict(X_test)

# Calcular métricas de avaliação do modelo
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error: {mae:.2f}")
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"r2 Score: {r2:.3f}")

Mean Absolute Error: 1513.13
Mean Squared Error: 3054250.36
Root Mean Squared Error: 1747.64
r2 Score: 0.900


## **5.2. Análise do Modelo**

A avaliação do modelo de Regressão Linear revelou um forte poder preditivo, com um **R² de 90.0%** e um **RMSE de $1,747.64**. Isso significa que o modelo explica 90% da variação no preço de revenda e suas previsões erram, em média, por este valor.

Contudo, a análise de diagnóstico confirmou a presença de **multicolinearidade severa**. Este fenômeno invalida a confiabilidade dos coeficientes do modelo, tornando impossível determinar o impacto individual de cada variável.

O modelo de Regressão Linear funciona como um excelente baseline de performance para previsões, mas **não é confiável para interpretação**. Ele acerta "o quê" (o valor previsto), mas falha em explicar "o porquê" (a importância de cada feature).

---

# **6. Modelos Ensemble**

Para superar as limitações de interpretabilidade da Regressão Linear, foram implementados dois modelos de *ensemble*:
* **Random Forest**
* **Gradient Boosting**.

Estes algoritmos baseados em árvores de decisão são naturalmente imunes à correlação entre features e capazes de capturar relações não-lineares complexas nos dados.

## **6.1. RandomForest**

O RandomForest é um poderoso algoritmo de *ensemble* que, como o nome sugere, constrói uma "floresta" de múltiplas árvores de decisão para chegar a um resultado.

Para mais detalhes sobre a implementação e seus parâmetros, consulte a [documentação oficial do Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html).


In [72]:
pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_jobs=-1, random_state=42))
])

pipeline_rf

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Battery_Capacity_kWh',
                                                   'Range_km',
                                                   'Charging_Power_kW',
                                                   'Charge_Cycles',
                                                   'Energy_Consumption_kWh_per_100km',
                                                   'Avg_Speed_kmh',
                                                   'Max_Speed_kmh',
                                                   'Acceleration_0_100_kmh_sec',
                                                   'Temperature_C',
                                                   'CO2_Saved_tons',
                                                   'Maintena...
                                                   'Electricity_Cost_USD_per_kWh',
                                                   'log_Charging_Time_hr',
                                                   'log_Monthly_Charging_Cost_USD',
                                                   'Vehicle_Age',
                                                   'Battery_Efficiency_km/kWh',
                                                   'Battery_Degradation_%',
                                                   'Avg_km_per_Year']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Make', 'Region',
                                                   'Vehicle_Type',
                                                   'Usage_Type'])])),
                ('regressor',
                 RandomForestRegressor(n_jobs=-1, random_state=42))])

In [73]:
# Treinar o pipeline completo com os dados de treino
pipeline_rf.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Battery_Capacity_kWh',
                                                   'Range_km',
                                                   'Charging_Power_kW',
                                                   'Charge_Cycles',
                                                   'Energy_Consumption_kWh_per_100km',
                                                   'Avg_Speed_kmh',
                                                   'Max_Speed_kmh',
                                                   'Acceleration_0_100_kmh_sec',
                                                   'Temperature_C',
                                                   'CO2_Saved_tons',
                                                   'Maintena...
                                                   'Electricity_Cost_USD_per_kWh',
                                                   'log_Charging_Time_hr',
                                                   'log_Monthly_Charging_Cost_USD',
                                                   'Vehicle_Age',
                                                   'Battery_Efficiency_km/kWh',
                                                   'Battery_Degradation_%',
                                                   'Avg_km_per_Year']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Make', 'Region',
                                                   'Vehicle_Type',
                                                   'Usage_Type'])])),
                ('regressor',
                 RandomForestRegressor(n_jobs=-1, random_state=42))])

In [74]:
# Fazer previsões nos dados de teste
y_pred_rf = pipeline_rf.predict(X_test)

# Calcular as métricas de avaliação
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Mean Absolute Error: {mae_rf:.2f}")
print(f"Mean Squared Error: {mse_rf:.2f}")
print(f"Root Mean Squared Error: {rmse_rf:.2f}")
print(f"r2 Score: {r2_rf:.3f}")

Mean Absolute Error: 1539.82
Mean Squared Error: 3293246.02
Root Mean Squared Error: 1814.73
r2 Score: 0.893


## **6.2. Análise**

O **Coeficiente de Determinação (R²)** de 89.3% indica que o modelo consegue explicar uma vasta maioria da variabilidade no valor de revenda, confirmando seu alto poder preditivo. O **RMSE** de aproximadamente $1,815 representa o erro médio de previsão do modelo em dólares.


## **6.3. GradientBoosting**
O Gradient Boosting é um poderoso algoritmo de *ensemble* que constrói árvores de decisão de forma sequencial, onde cada nova árvore é treinada para corrigir os erros da anterior.

Para mais detalhes sobre a implementação e seus parâmetros, consulte a [documentação oficial do Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html).

In [75]:
pipeline_gb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(random_state=42))
])
pipeline_gb

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Battery_Capacity_kWh',
                                                   'Range_km',
                                                   'Charging_Power_kW',
                                                   'Charge_Cycles',
                                                   'Energy_Consumption_kWh_per_100km',
                                                   'Avg_Speed_kmh',
                                                   'Max_Speed_kmh',
                                                   'Acceleration_0_100_kmh_sec',
                                                   'Temperature_C',
                                                   'CO2_Saved_tons',
                                                   'Maintena...
                                                   'Electricity_Cost_USD_per_kWh',
                                                   'log_Charging_Time_hr',
                                                   'log_Monthly_Charging_Cost_USD',
                                                   'Vehicle_Age',
                                                   'Battery_Efficiency_km/kWh',
                                                   'Battery_Degradation_%',
                                                   'Avg_km_per_Year']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Make', 'Region',
                                                   'Vehicle_Type',
                                                   'Usage_Type'])])),
                ('regressor', GradientBoostingRegressor(random_state=42))])

In [76]:
# Treinar o pipeline completo com os dados de treino
pipeline_gb.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Battery_Capacity_kWh',
                                                   'Range_km',
                                                   'Charging_Power_kW',
                                                   'Charge_Cycles',
                                                   'Energy_Consumption_kWh_per_100km',
                                                   'Avg_Speed_kmh',
                                                   'Max_Speed_kmh',
                                                   'Acceleration_0_100_kmh_sec',
                                                   'Temperature_C',
                                                   'CO2_Saved_tons',
                                                   'Maintena...
                                                   'Electricity_Cost_USD_per_kWh',
                                                   'log_Charging_Time_hr',
                                                   'log_Monthly_Charging_Cost_USD',
                                                   'Vehicle_Age',
                                                   'Battery_Efficiency_km/kWh',
                                                   'Battery_Degradation_%',
                                                   'Avg_km_per_Year']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Make', 'Region',
                                                   'Vehicle_Type',
                                                   'Usage_Type'])])),
                ('regressor', GradientBoostingRegressor(random_state=42))])

In [77]:
# Fazer previsões nos dados de teste
y_pred_gb = pipeline_gb.predict(X_test)

# Calcular as métricas de avaliação
mae_gb = mean_absolute_error(y_test, y_pred_gb)
mse_gb = mean_squared_error(y_test, y_pred_gb)
rmse_gb = np.sqrt(mse_gb)
r2_gb = r2_score(y_test, y_pred_gb)
print(f"Mean Absolute Error: {mae_gb:.2f}")
print(f"Mean Squared Error: {mse_gb:.2f}")
print(f"Root Mean Squared Error: {rmse_gb:.2f}")
print(f"r2 Score: {r2_gb:.3f}")

Mean Absolute Error: 1513.02
Mean Squared Error: 3104039.51
Root Mean Squared Error: 1761.83
r2 Score: 0.899


## **6.4. Análise**

Com um **Coeficiente de Determinação (R²) de 89.9%**, o Gradient Boosting praticamente igualou a performance preditiva da Regressão Linear, estabelecendo-se como o modelo mais acurado. O **MAE e o RMSE** são os mais baixos entre os modelos de ensemble, confirmando sua superioridade em minimizar os erros de previsão.

> O Gradient Boosting se firma como o **melhor modelo para este projeto**, oferecendo previsões mais precisas.

---

# **7. Feature Importance**

Após treinar e avaliar os modelos, a etapa final consiste em interpretar seus resultados para responder à pergunta central do projeto:
> "Quais são as características que mais influenciam o valor de revenda de um veículo elétrico?"

Ao comparar a importância das features entre os dois modelos de ensemble, busca-se identificar um padrão consistente que revele os verdadeiros impulsionadores de valor no dataset, transformando os resultados do modelo em insights acionáveis.

In [78]:
# Acessa a etapa 'preprocessor' do pipeline e obtém os nomes das colunas
# após a aplicação do OneHotEncoder e do StandardScaler.
feature_names = pipeline_rf.named_steps['preprocessor'].get_feature_names_out()

# Acessa a etapa 'regressor' do pipeline e, em seguida, o atributo .feature_importances_
rf_importances = pipeline_rf.named_steps['regressor'].feature_importances_

# Cria um DataFrame para visualizar as features mais importantes
rf_feature_importance_df = pd.DataFrame(
    {'feature': feature_names, 'importance': rf_importances.round(3)}
).sort_values('importance', ascending=False)

rf_feature_importance_df.head(10)


,feature,importance
0,num__Battery_Capacity_kWh,0.851
15,num__Vehicle_Age,0.057
5,num__Avg_Speed_kmh,0.006
4,num__Energy_Consumption_kWh_per_100km,0.006
8,num__Temperature_C,0.006
10,num__Maintenance_Cost_USD,0.006
6,num__Max_Speed_kmh,0.005
2,num__Charging_Power_kW,0.005
17,num__Battery_Degradation_%,0.005
7,num__Acceleration_0_100_kmh_sec,0.005


In [79]:
gb_importances = pipeline_gb.named_steps['regressor'].feature_importances_

# Cria o DataFrame para o Gradient Boosting
gb_feature_importance_df = pd.DataFrame(
    {'feature': feature_names, 'importance': gb_importances.round(3)}
).sort_values('importance', ascending=False)

gb_feature_importance_df.head(10)

,feature,importance
0,num__Battery_Capacity_kWh,0.911
15,num__Vehicle_Age,0.067
1,num__Range_km,0.005
4,num__Energy_Consumption_kWh_per_100km,0.002
8,num__Temperature_C,0.002
16,num__Battery_Efficiency_km/kWh,0.002
3,num__Charge_Cycles,0.001
2,num__Charging_Power_kW,0.001
7,num__Acceleration_0_100_kmh_sec,0.001
6,num__Max_Speed_kmh,0.001


# **8. Conclusão**

Ambos os modelos, apesar de suas abordagens algorítmicas distintas, aprenderam a mesma lição fundamental a partir dos dados.

1.  **A Bateria é Soberana:** Em ambos os modelos, a **`Battery_Capacity_kWh`** não é apenas a feature mais importante; ela é **dominantemente** mais importante. Com uma importância de **85% (Random Forest)** e **91% (Gradient Boosting)**, ela ofusca todas as outras variáveis combinadas.
    * A capacidade da bateria é o principal fator técnico que dita o custo do veículo. Portanto, para prever o preço, a primeira pergunta a se fazer é: "Qual o tamanho da bateria?".

2.  **Depreciação como Fator Secundário:** A segunda variável mais importante em ambos os modelos é a `Vehicle_Age`, com cerca de 6-7% de importância.
    * Este resultado destaca o sucesso da etapa de engenharia de features. Ao transformar a coluna `Year` na variável `Vehicle_Age`, foi fornecido ao modelo um indicador direto e poderoso para o conceito de depreciação.

**As Demais Features Têm Impacto Marginal:** Todas as outras dezenas de features, incluindo as que foram criadas, apresentaram uma importância quase insignificante (abaixo de 1%), indicando que a maior parte do poder preditivo estava concentrada nas duas primeiras variáveis.

---